<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Telecomm EDA Challenge Lab

_Author: Alex Combs (NYC) _

---

Let's do some Exploratory Data Analysis (EDA)! As a data scientist, you often may find yourself given a data set you've never seen before, and asked to do a rapid analysis. This is today's goal.

# Prompt

You work for a telecommunications company. The company has been storing metadata about customer phone usage, as part of the regular course of business. Currently, this data is sitting in an unsecured database. The company doesn't want to pay to increase their database security, because they don't think there's really anything to be learned from the metadata.

They are under pressure from "right to privacy" organizations to beef up the database security. These organizations argue that you can learn a lot about a person from their cell phone metadata.

The telecom company wants to understand if this is true, and they want your help. They will give you one person's metadata for 2014 and want to see what you can learn from it.

Working in teams, create a report revealing everything you can about the person. Prepare a presentation, with slides, showcasing your findings.


# The Data

The [person's metadata](./datasets/metadata.csv) has the following fields:

| Field Name          | Description
| ---                 | ---
| **Cell Cgi**        | cell phone tower identifier
| **Cell Tower**      | cell phone tower location
| **Comm Identifier** |	de-identified recipient of communication
| **Comm Timedate String** | time of communication
| **Comm Type	Id**  | type of communication
| **Latitude**        | latitude of communication
| **Longitude**       | longitude of communication


# Hints

This is totally open-ended! If you're totally stumped -- and only if stumped -- should you look below for prompts. As a starting point, given that you have geo-locations, consider investigating ways to display this type of information (i.e. mapping functionality).

<font color='white'>
Well for starters, he's in Australia!

Ideas for things to look into:
- where does he work?
- where does he live?
- who does he contact most often?
- what hours does he work?
- did he move?
- did he go on holiday?  If so, where did he go?
- did he get a new phone?

Challenges:
- how does he get to work?
- where does his family live?
- if he went on holiday, can you find which flights he took?
- can you guess who some of his contacts are, based on the frequency, location, time and mode (phone/text) of communications?


If you're stuck on how to map the data, you can try "basemap" or "gmplot", or anything else you find online.
</font>

In [1]:
#Install geopy
!conda install folium --yes
!pip install folium
!pip install geopy

Solving environment: ...working... failed



PackagesNotFoundError: The following packages are not available from current channels:

  - folium

Current channels:

  - https://repo.continuum.io/pkgs/main/win-64
  - https://repo.continuum.io/pkgs/main/noarch
  - https://repo.continuum.io/pkgs/free/win-64
  - https://repo.continuum.io/pkgs/free/noarch
  - https://repo.continuum.io/pkgs/r/win-64
  - https://repo.continuum.io/pkgs/r/noarch
  - https://repo.continuum.io/pkgs/pro/win-64
  - https://repo.continuum.io/pkgs/pro/noarch
  - https://repo.continuum.io/pkgs/msys2/win-64
  - https://repo.continuum.io/pkgs/msys2/noarch




  Running setup.py bdist_wheel for folium: started
  Running setup.py bdist_wheel for folium: finished with status 'done'
  Stored in directory: C:\Users\matin\AppData\Local\pip\Cache\wheels\04\d0\a0\b2b8356443364ae79743fce0b9b6a5b045f7560742129fde22
Successfully built folium


In [3]:
!pip install gpxpy

  Running setup.py bdist_wheel for gpxpy: started
  Running setup.py bdist_wheel for gpxpy: finished with status 'done'
  Stored in directory: C:\Users\matin\AppData\Local\pip\Cache\wheels\1b\b4\7c\c4775d65f2115b055c2cd6cbf4e4cac5bd4f5eb680d00db4f1
Successfully built gpxpy


In [4]:
#new packages
import gpxpy.geo as gg
import geopy as ge
import folium as fo

#import other packages
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [10]:
df = pd.read_csv("./datasets/metadata.csv")

In [11]:
df.head()

,Cell Cgi,Cell Tower Location,Comm Identifier,Comm Timedate String,Comm Type,Latitude,Longitude
0,50501015388B9,REDFERN TE,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 9:40,Phone,-33.892933,151.202296
1,50501015388B9,REDFERN TE,62157ccf2910019ffd915b11fa037243b75c1624,4/1/14 9:42,Phone,-33.892933,151.202296
2,505010153111F,HAYMARKET #,c8f92bd0f4e6fb45ed7fce96fc831b283db2b642,4/1/14 13:13,Phone,-33.880329,151.205690
3,505010153111F,HAYMARKET #,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 13:13,Phone,-33.880329,151.205690
4,5.05E+106,HAYMARKET #,f1a6836c0b7a3415a19a90fdd6f0ae18484d6d1e,4/1/14 17:27,Phone,-33.880329,151.205690


In [12]:
df.shape

(10476, 7)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10476 entries, 0 to 10475
Data columns (total 7 columns):
Cell Cgi                10476 non-null object
Cell Tower Location     10476 non-null object
Comm Identifier         1374 non-null object
Comm Timedate String    10476 non-null object
Comm Type               10476 non-null object
Latitude                10476 non-null float64
Longitude               10476 non-null float64
dtypes: float64(2), object(5)
memory usage: 573.0+ KB


In [15]:
df.isnull().sum()

Cell Cgi                   0
Cell Tower Location        0
Comm Identifier         9102
Comm Timedate String       0
Comm Type                  0
Latitude                   0
Longitude                  0
dtype: int64

In [20]:
internet = df.loc[df["Comm Identifier"].isnull()]
internet.shape

(9102, 7)

In [21]:
internet["Comm Type"].unique()

array(['Internet'], dtype=object)

In [23]:
phone = df.loc[df["Comm Identifier"].notnull()]
phone.shape

(1374, 7)

In [24]:
phone["Comm Type"].unique()

array(['Phone', 'SMS'], dtype=object)

### Observations:

- We can assume based on telecomm data that this individual has been in Australia from April 2014 - 2015 
- Telecomm data from Australia based on location of Cell Towers
    - New South Wales, Victoria, and Tasmania
    - Most data points in Sydney
- Found 3 Types of "Comm Type" data:  Phone, SMS, Internet


In [31]:
df["Cell Tower Location"].value_counts()

BALGOWLAH HAYES ST                          4301
CHIPPENDALE                                 1084
SUNDERLAND ST                                723
REDFERN TE                                   712
HAYMARKET #                                  563
BRICKWORKS                                   501
HARBORD 22 WAINE ST                          465
FAIRLIGHT 137 SYDNEY RD                      454
MANLY #                                      231
NEW TOWN                                     197
CHINATOWN                                    161
BEECHWORTH                                   112
BALGOWLAH VILLAGE SHOPPING CENTRE IBC        106
MANLY SOUTH STEYNE                            92
BROADWAY OTC                                  85
MASCOT INTERNATIONAL AIRPORT TERMINAL T1      65
71 MACQUARIE ST                               49
SURRY HILLS 418A ELIZABETH ST                 45
MANLY NTH STEYNE                              40
MASCOT M5 MOTORWAY EMERGENCY STAIRS           33
BALGOWLAH TE        

In [32]:
df.describe()

,Latitude,Longitude
count,10476.000000,10476.000000
mean,-35.136188,150.612841
std,3.141723,1.470169
min,-42.884810,144.848243
25%,-33.884603,151.202296
50%,-33.796610,151.266540
75%,-33.788150,151.266540
max,-33.557310,151.289340


In [54]:
#'Cell Tower Location' - Location of 	Comm Identifier	Comm Timedate String	Comm Type	Latitude	Longitude

a = df["Latitude"].unique()
b = df["Longitude"].unique()
c = pd.DataFrame(zip(a, b))

In [62]:
len(c)

70

In [67]:
c.iloc[1,0]

-33.880328910000003

In [73]:
map_location = fo.Map(location=[-35.136188, 150.612841], zoom_start = 5)

for x in range(len(c)):
    fo.Marker([c.iloc[x,0], c.iloc[x,1]], popup='Mt. Hood Meadows').add_to(map_location)
map_location

In [88]:
d = zip(df["Latitude"], df["Longitude"])

In [89]:
from folium.plugins import HeatMap
map2 = fo.Map(location=[-35.136188, 150.612841], zoom_start = 5)

In [91]:
HeatMap(d).add_to(map2)
map2